In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("SmolLM2-1.7B-Instruct-ov-INT4")
chat = [
    {'role': 'user', 'content': "hello"}
]

print(tokenizer.apply_chat_template(chat, tokenize=False))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
hello<|im_end|>



In [1]:
from langchain_community.embeddings import OpenVINOEmbeddings
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFacePipeline
from prompt import prompt_qwen
from langchain_community.document_compressors.openvino_rerank import OpenVINOReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain


model_kwargs = {"device": "CPU"}
encode_kwargs = {"normalize_embeddings": True}
gen_config = {"skip_prompt": True, "pipeline_kwargs": {"max_new_tokens": 512}}
ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name_or_path" in OpenVINOReranker has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in OpenVINOReranker has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
embed_model = OpenVINOEmbeddings(
    model_name_or_path="srang992/all-mpnet-base-v2-ov",
    model_kwargs=model_kwargs,
    encode_kwargs = encode_kwargs
)

ov_llm = HuggingFacePipeline.from_model_id(
    model_id="srang992/Qwen-2.5-3B-Instruct-ov-INT4",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "CPU", 
                  "ov_config": ov_config, 
                  "temperature": 0.0, 
                  "max_length": 512,
                  "repetition_penalty": 1.178},
)

reranker = OpenVINOReranker(
    model_name_or_path="llmware/jina-reranker-v1-turbo-en-ov",
    model_kwargs={"device": "CPU"},
    top_n=2,
)

vectorstore = FAISS.load_local("vectorstore", embed_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
retriever_with_rerank = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=retriever)
llm = ov_llm.bind(**gen_config)
qna_chain = create_stuff_documents_chain(llm, prompt_qwen)
rag_chain = create_retrieval_chain(retriever=retriever_with_rerank, combine_docs_chain=qna_chain)

chain = rag_chain.pick("answer")
def gen_answer(question):
    for chunk in chain.stream({"input": question}):
        print(f"{chunk}", end="")

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

openvino_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

openvino_model.xml:   0%|          | 0.00/432k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/854k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# gen_answer("list every types of machine learning.")

In [3]:
gen_answer("what are the types of machine learning? Explain each of them.")

Machine learning systems can be broadly classified into several categories based on different criteria. Here's an explanation of each category:

### Supervised Learning
In supervised learning, the algorithm learns from labeled data. This means that each example in the dataset has both features and the corresponding label. The goal is to predict the correct label for new, unseen data based on what it learned from the labeled examples.

#### Examples:
- Image classification: Training a model to recognize cats, dogs, etc., by showing it images with labels.
- Spam detection: Training a model to identify emails as spam or not spam by providing a dataset of emails with their respective labels.

### Unsupervised Learning
Unsupervised learning involves training models on unlabeled data. The algorithm tries to find hidden patterns or intrinsic structures within the data without prior knowledge of how the data should be classified.

#### Examples:
- Clustering: Grouping similar items together, s

In [7]:
gen_answer("tell me about aurelien geron?")

In [ ]:
gen_answer("what is online learning? Explain.")

In [ ]:
gen_answer("what is adaboost classifier?")

In [ ]:
gen_answer("what is overfitting? Explain it in simple way.")

In [9]:
# gen_answer("when to use dropout and when to use regularization?")

In [ ]:
gen_answer("How to prevent overfitting?")

In [ ]:
# gen_answer("Explain overfitting in simple way.")

In [ ]:
gen_answer("What is L1 regularization? Explain it in a simple way.")

In [ ]:
gen_answer("what is ridge regression?")

In [ ]:
gen_answer("What is Random forest?")

In [ ]:
gen_answer("what is feature importance in random forest?")

In [ ]:
gen_answer("what is RNN?")